# The Simplified Whiskas Model Python Formulation for the PuLP Modeller

#### Authors: Antony Phillips, Dr Stuart Mitchell  2007

### Problem Description:

Whiskas cat food is manufactured by Uncle Ben’s. Uncle Ben’s want to produce their cat food products as cheaply as possible while ensuring they meet the stated nutritional analysis requirements shown on the cans. Thus they want to vary the quantities of each ingredient used (the main ingredients being chicken, beef, mutton, rice, wheat and gel) while still meeting their nutritional standards. The costs of the chicken, beef, and mutton are \$0.013, \$0.008 and \$0.010 respectively, while the costs of the rice, wheat and gel are \$0.002, \$0.005 and \$0.001 respectively (all costs are per gram). For this exercise we will ignore the vitamin and mineral ingredients, any costs for these are likely to be very small anyway.

Each ingredient contributes to the total weight of protein, fat, fiber and salt in the final product. The contributions per gram of ingredient are given in the table below.

|Ingredient|Protein|Fat   |Fibre |Salt |
|----------|-------|------|------|-----|
|Chicken   |0.100  |0.080 |0.001 |0.002|
|Beef      |0.200  |0.100 |0.005 |0.005|
|Mutton	   |0.150  |0.110 |0.003 |0.007|
|Rice      |0.000  |0.010 |0.100 |0.002|
|Wheat Bran|0.040  |0.010 |0.150 |0.008|
*Ingredient's contribution (in grams)

To meet the nutritional analysis requirements, we need to have at least 8g of Protein per 100g, 6g of fat, but no more than 2g of fiber and 0.4g of salt. Formulate the LP model that minimizes cost per can of cat food.


In [1]:
# pip install pulp
from pulp import *

In [2]:
# List of the Ingredients
ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL']

In [3]:
# A dictionary of the costs of each of the Ingredients
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001}

# A dictionary of the protein percent in each of the Ingredients
protein_percent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000}

# A dictionary of the fat percent in each of the Ingredients
fat_percent =     {'CHICKEN': 0.080, 
                  'BEEF': 0.100, 
                  'MUTTON': 0.110, 
                  'RICE': 0.010, 
                  'WHEAT': 0.010, 
                  'GEL': 0.000}

# A dictionary of the fibre percent in each of the Ingredients
fibre_percent =   {'CHICKEN': 0.001, 
                  'BEEF': 0.005, 
                  'MUTTON': 0.003, 
                  'RICE': 0.100, 
                  'WHEAT': 0.150, 
                  'GEL': 0.000}

# A dictionary of the salt percent in each of the Ingredients
salt_percent =    {'CHICKEN': 0.002, 
                  'BEEF': 0.005, 
                  'MUTTON': 0.007, 
                  'RICE': 0.002, 
                  'WHEAT': 0.008, 
                  'GEL': 0.000}

In [4]:
# prob variable to hold the problem data
prob = LpProblem('The Whiskas Problem', LpMinimize)

In [5]:
# A dictionary to hold the referenced Variables
ingredient_vars = LpVariable.dicts('Ingr', ingredients, 0)
ingredient_vars

{'BEEF': Ingr_BEEF,
 'CHICKEN': Ingr_CHICKEN,
 'GEL': Ingr_GEL,
 'MUTTON': Ingr_MUTTON,
 'RICE': Ingr_RICE,
 'WHEAT': Ingr_WHEAT}

In [6]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i] * ingredient_vars[i] for i in ingredients]), 'Total cost of ingredients per can'

In [7]:
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in ingredients]) == 100, 'Percentages Sum'
prob += lpSum([protein_percent[i] * ingredient_vars[i] for i in ingredients]) >= 8.0, 'Protein requirement'
prob += lpSum([fat_percent[i] * ingredient_vars[i] for i in ingredients]) >= 6.0, 'Fat requirement'
prob += lpSum([fibre_percent[i] * ingredient_vars[i] for i in ingredients]) <= 2.0, 'Fibre requirement'
prob += lpSum([salt_percent[i] * ingredient_vars[i] for i in ingredients]) <= 0.4, 'Salt requirement'

In [8]:
# Uncomment below line to save the problem data to an .lp file
# prob.writeLP("Whiskas_big.lp")
prob

The Whiskas Problem:
MINIMIZE
0.008*Ingr_BEEF + 0.013*Ingr_CHICKEN + 0.001*Ingr_GEL + 0.01*Ingr_MUTTON + 0.002*Ingr_RICE + 0.005*Ingr_WHEAT + 0.0
SUBJECT TO
Percentages_Sum: Ingr_BEEF + Ingr_CHICKEN + Ingr_GEL + Ingr_MUTTON + Ingr_RICE
 + Ingr_WHEAT = 100

Protein_requirement: 0.2 Ingr_BEEF + 0.1 Ingr_CHICKEN + 0.15 Ingr_MUTTON
 + 0.04 Ingr_WHEAT >= 8

Fat_requirement: 0.1 Ingr_BEEF + 0.08 Ingr_CHICKEN + 0.11 Ingr_MUTTON
 + 0.01 Ingr_RICE + 0.01 Ingr_WHEAT >= 6

Fibre_requirement: 0.005 Ingr_BEEF + 0.001 Ingr_CHICKEN + 0.003 Ingr_MUTTON
 + 0.1 Ingr_RICE + 0.15 Ingr_WHEAT <= 2

Salt_requirement: 0.005 Ingr_BEEF + 0.002 Ingr_CHICKEN + 0.007 Ingr_MUTTON
 + 0.002 Ingr_RICE + 0.008 Ingr_WHEAT <= 0.4

VARIABLES
Ingr_BEEF Continuous
Ingr_CHICKEN Continuous
Ingr_GEL Continuous
Ingr_MUTTON Continuous
Ingr_RICE Continuous
Ingr_WHEAT Continuous

In [9]:
%%time
# The problem is solved using PuLP's choice of Solver
prob.solve()

Wall time: 748 ms


1

In [10]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [11]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Ingr_BEEF = 60.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 40.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0


In [12]:
# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = ", value(prob.objective))

Total Cost of Ingredients per can =  0.52


#### End of first Git Jupyter Notebook